In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

# GOES Flare Profiles for SolarNN

## Flare Profiles - Case 1
- Flare profiles determined by GOES detections and random intervals between GOES detections.

In [ ]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_wave_v2.csv", index=False)

Profiles_df.head(), Profiles_df.shape

## Flare Profiles - Case 2
- Flare profiles determined by GOES detections and random intervals between wavelet detections.

In [3]:
# Step 1: Read the CSV file
flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")

# Step 2: Identify flare and wavelet indices
flare_indices = flares2017_2022.index[flares2017_2022['Flare'] == 1].tolist()
wavelet_indices = flares2017_2022.index[flares2017_2022['Wavelet'] == 1].tolist()

numFlares = len(flare_indices)

# Step 3: Initialize parameters and matrices
hist1 = 15  # mins before
hist2 = 15  # mins after
nonflareMultiple = 20  # how many non-flares to generate for every flare
cols = hist1 + hist2 + 1 + 1 + 1

FlareProfiles = np.empty((numFlares, cols))
FlareProfiles[:] = np.nan

j = 0
for i in flare_indices:
    if i > hist1 and i <= len(flares2017_2022) - hist2:
        fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
        FlareProfiles[j, :-2] = fluxes.values
        FlareProfiles[j, -2] = 1
        FlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
        j += 1

# Step 4: Generate non-flare profiles
exclude_indices = set(wavelet_indices)
for k in range(1, 6):
    exclude_indices.update([idx + k for idx in wavelet_indices])
    exclude_indices.update([idx - k for idx in wavelet_indices])

nonflare_indices = set(range(hist1 + 1, len(flares2017_2022) - (hist2 + 1))) - exclude_indices
nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)

NotFlareProfiles = np.empty((nonflareMultiple * numFlares, cols))
NotFlareProfiles[:] = np.nan

j = 0
for i in nonflare_indices:
    fluxes = flares2017_2022['Flux'].iloc[(i - hist1):(i + hist2 + 1)] / flares2017_2022['Flux'].iloc[i]
    NotFlareProfiles[j, :-2] = fluxes.values
    NotFlareProfiles[j, -2] = 0
    NotFlareProfiles[j, -1] = flares2017_2022['secs_since_2000'].iloc[i]
    j += 1

# Step 5: Combine and save the profiles
Profiles = np.vstack([FlareProfiles, NotFlareProfiles])
Profiles_df = pd.DataFrame(Profiles, columns=[f'x{i+1}' for i in range(hist1 + hist2 + 1)] + ['y', 'SecsSince2000'])
Profiles_df = Profiles_df.dropna()

# Convert 'SecsSince2000' to datetime
Profiles_df['Date'] = pd.to_datetime(Profiles_df['SecsSince2000'], unit='s', origin=pd.Timestamp('2000-01-01 12:00:00'))
Profiles_df.drop('SecsSince2000', axis=1, inplace=True)

# Save to CSV
Profiles_df.to_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/Profiles2017_2022_wave_v2.csv", index=False)

Profiles_df.head(), Profiles_df.shape

/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/2923137786.py:2: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  flares2017_2022 = pd.read_csv("/Users/joshuaingram/Main/Projects/SolarFlareNN/data/FlaresGoes161718Wavelet_2.csv")
/var/folders/k3/6x6tz2rj1qb4727bvm_ytpqc0000gn/T/ipykernel_50021/2923137786.py:35: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  nonflare_indices = random.sample(nonflare_indices, nonflareMultiple * numFlares)


(         x1        x2        x3        x4        x5        x6        x7  \
 0  0.249173  0.278074  0.289489  0.297877  0.323917  0.323846  0.293258   
 1  0.185302  0.195949  0.204914  0.203649  0.217488  0.225099  0.233949   
 2  0.071275  0.084802  0.110699  0.211815  0.258221  0.193603  0.142327   
 3  0.722010  0.677981  0.669214  0.650412  0.641111  0.633326  0.625018   
 4  0.729120  0.716180  0.701948  0.645696  0.609369  0.585797  0.576336   
 
          x8        x9       x10  ...       x24       x25       x26       x27  \
 0  0.300184  0.320395  0.401313  ...  0.600410  0.576188  0.541754  0.507117   
 1  0.248383  0.258860  0.267442  ...  0.877201  1.928971  3.298110  3.872649   
 2  0.106506  0.087165  0.077384  ...  0.351790  0.349945  0.343246  0.341172   
 3  0.647180  0.679063  0.713814  ...  0.766183  0.723078  0.695107  0.683881   
 4  0.560038  0.550306  0.534466  ...  1.558642  2.091865  2.234191  2.047342   
 
         x28       x29       x30       x31    y       

## Flare Profiles - Case 3
- Flare profiles for positive cases determined by GOES detections, and slide center point of window over full duration of flare. Negative cases determined by wavelet detection

## Flare Profiles - Case 4
- Flare profiles determined by GOES detections and random intervals between wavelet detections. Do not normalize flux values.

## Flare Profiles - Case 5
- Flare profiles determined by GOES detections and random intervals between wavelet detections. Change time scales of flare profiles.

## Flare Profiles - Case 5 
- Flare profiles determined by GOES detections, random intervals between wavelet detections, and random noise at sub A-class flux levels.